In [9]:
import numpy as np
import re
from collections import defaultdict


# Step 1: Create sample documents about Messi and Ronaldo with higher similarity
documents = [
    # Messi documents (with repeated phrases and similarities)
    "Lionel Messi is widely regarded as the greatest footballer of all time. His ball control and vision are unmatched.",
    "Lionel Messi is widely regarded as the greatest footballer of all time. His technique and ball control define his game.",
    "The Argentine maestro Messi has won a record 8 Ballon d'Or awards, cementing his legacy as the greatest footballer.",
    "Messi led Argentina to World Cup glory in 2022, completing his trophy collection with international success.",
    "Barcelona's golden era was defined by Messi's magical performances and his unmatched ball control and vision.",

    # Ronaldo documents (with repeated phrases and similarities)
    "Cristiano Ronaldo is known for his physical prowess and incredible goal-scoring abilities in the world of football.",
    "Cristiano Ronaldo is known for his work ethic and physical prowess that made him one of the greatest footballers.",
    "The Portuguese forward has won 5 Champions League titles during his illustrious career in European football.",
    "Ronaldo's training regimen and dedication have extended his career at the top level of football for many years.",
    "CR7 has scored over 800 goals in his professional career, demonstrating his incredible goal-scoring abilities.",

    # Comparison documents (with more shared phrases from both players)
    "While Ronaldo excels at goal-scoring, Messi combines goal-scoring with unparalleled vision and ball control.",
    "Messi's natural talent and Ronaldo's work ethic represent two paths to becoming the greatest footballer of all time.",
    "Ronaldo's physical prowess and Messi's ball control define their different playing styles in world football.",
    "Both have incredible goal-scoring abilities, but Messi's vision and Ronaldo's physical attributes set them apart.",
    "Lionel Messi and Cristiano Ronaldo have defined an era of football with their trophy collections and achievements."
]

# Step 2: Create shingles from documents
def create_shingles(text, k=2):
    """Convert text into k-shingles (k consecutive words)"""
    # Preprocess text: lowercase and remove punctuation
    text = re.sub(r'[^\w\s]', '', text.lower())
    words = text.split()
    # Create k-shingles
    return set([' '.join(words[i:i+k]) for i in range(len(words)-k+1)])

# Create shingles for each document
shingles_per_doc = [create_shingles(doc) for doc in documents]

# Create a universal set of all shingles
all_shingles = set()
for shingles in shingles_per_doc:
    all_shingles.update(shingles)

# Map each shingle to an index for our signature matrix
shingle_to_idx = {shingle: i for i, shingle in enumerate(all_shingles)}

# Step 3: Create document vectors as binary arrays (1 if shingle present, 0 if not)
def create_doc_vector(doc_shingles):
    """Create a binary vector for a document based on its shingles"""
    vector = np.zeros(len(all_shingles), dtype=np.int8)
    for shingle in doc_shingles:
        vector[shingle_to_idx[shingle]] = 1
    return vector

doc_vectors = [create_doc_vector(doc_shingles) for doc_shingles in shingles_per_doc]

# Step 4: MinHash implementation
def generate_hash_functions(n, max_val):
    """Generate n hash functions of the form (ax + b) % c"""
    # Using prime numbers for coefficients
    hash_functions = []
    for _ in range(n):
        a = np.random.randint(1, max_val)
        b = np.random.randint(0, max_val)
        hash_functions.append((a, b, max_val))
    return hash_functions

def minhash_signature(doc_vector, hash_functions):
    """Generate a minhash signature for a document"""
    signature = np.full(len(hash_functions), np.inf)

    # For each row where document has a 1
    for row in np.where(doc_vector == 1)[0]:
        # Apply each hash function
        for i, (a, b, c) in enumerate(hash_functions):
            hash_val = (a * row + b) % c
            signature[i] = min(signature[i], hash_val)

    return signature

# Create MinHash signatures
num_hashes = 100  # Number of hash functions
hash_functions = generate_hash_functions(num_hashes, len(all_shingles)*10)
signatures = [minhash_signature(doc_vector, hash_functions) for doc_vector in doc_vectors]

# Step 5: LSH implementation
def lsh_index(signatures, bands, rows):
    """Create LSH index for fast similarity search"""
    assert bands * rows == len(signatures[0]), "bands * rows must equal signature length"

    # Initialize buckets
    buckets = defaultdict(list)

    # For each band
    for band in range(bands):
        # For each document
        for doc_id, signature in enumerate(signatures):
            # Get the portion of signature for this band
            band_signature = tuple(signature[band * rows: (band + 1) * rows])
            # Add document to the bucket
            buckets[(band, band_signature)].append(doc_id)

    return buckets

def find_candidate_pairs(buckets):
    """Find candidate pairs of similar documents from LSH buckets"""
    candidate_pairs = set()

    for docs in buckets.values():
        # If bucket has at least 2 documents
        if len(docs) >= 2:
            # Add all pairs of documents in this bucket
            for i in range(len(docs)):
                for j in range(i+1, len(docs)):
                    candidate_pairs.add(tuple(sorted([docs[i], docs[j]])))

    return candidate_pairs

# Apply LSH
bands = 20
rows = num_hashes // bands
lsh_buckets = lsh_index(signatures, bands, rows)
candidate_pairs = find_candidate_pairs(lsh_buckets)

# Step 6: Calculate Jaccard similarity for candidate pairs
def jaccard_similarity(set1, set2):
    """Calculate Jaccard similarity between two sets"""
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

# Compute and display similar document pairs
print("LSH and MinHash Demo with Messi vs. Ronaldo Texts\n")
print("Our documents:")
for i, doc in enumerate(documents):
    print(f"{i}. {doc[:80]}...")
print("\n")

print("Similar document pairs found by LSH:")
for doc1_id, doc2_id in candidate_pairs:
    similarity = jaccard_similarity(shingles_per_doc[doc1_id], shingles_per_doc[doc2_id])
    if similarity > 0.2:  # Filter out very low similarities
        print(f"Documents {doc1_id} and {doc2_id} - Similarity: {similarity:.2f}")
        print(f"  - {documents[doc1_id]}")
        print(f"  - {documents[doc2_id]}")
        print()

# Step 7: Analysis of Messi-praising documents
print("Messi Favoritism Analysis:")
messi_docs = [0, 1, 2, 3, 4]  # Indices of Messi documents
ronaldo_docs = [5, 6, 7, 8, 9]  # Indices of Ronaldo documents
comparison_docs = [10, 11, 12, 13, 14]  # Indices of comparison documents

print("\nSimilarities between Messi-praising documents:")
for i in range(len(messi_docs)):
    for j in range(i+1, len(messi_docs)):
        doc1_id, doc2_id = messi_docs[i], messi_docs[j]
        similarity = jaccard_similarity(shingles_per_doc[doc1_id], shingles_per_doc[doc2_id])
        print(f"Messi docs {doc1_id} and {doc2_id}: {similarity:.2f}")

print("\nSimilarities between comparison documents (favoring Messi) and Messi documents:")
for comp_id in comparison_docs:
    for messi_id in messi_docs:
        similarity = jaccard_similarity(shingles_per_doc[comp_id], shingles_per_doc[messi_id])
        print(f"Comparison doc {comp_id} and Messi doc {messi_id}: {similarity:.2f}")

print("\nSimilarities between comparison documents (favoring Messi) and Ronaldo documents:")
for comp_id in comparison_docs:
    for ronaldo_id in ronaldo_docs:
        similarity = jaccard_similarity(shingles_per_doc[comp_id], shingles_per_doc[ronaldo_id])
        print(f"Comparison doc {comp_id} and Ronaldo doc {ronaldo_id}: {similarity:.2f}")

LSH and MinHash Demo with Messi vs. Ronaldo Texts

Our documents:
0. Lionel Messi is widely regarded as the greatest footballer of all time. His ball...
1. Lionel Messi is widely regarded as the greatest footballer of all time. His tech...
2. The Argentine maestro Messi has won a record 8 Ballon d'Or awards, cementing his...
3. Messi led Argentina to World Cup glory in 2022, completing his trophy collection...
4. Barcelona's golden era was defined by Messi's magical performances and his unmat...
5. Cristiano Ronaldo is known for his physical prowess and incredible goal-scoring ...
6. Cristiano Ronaldo is known for his work ethic and physical prowess that made him...
7. The Portuguese forward has won 5 Champions League titles during his illustrious ...
8. Ronaldo's training regimen and dedication have extended his career at the top le...
9. CR7 has scored over 800 goals in his professional career, demonstrating his incr...
10. While Ronaldo excels at goal-scoring, Messi combines goal-sc